# Interacción con las campañas <a class="anchor" id="top"></a>

En este cuaderno, implementará e interactuará con campañas en Amazon Personalize.

1. [Introducción](#intro)
1. [Crear campañas](#create)
1. [Interactuar con campañas](#interact)
1. [Recomendaciones por lote](#batch)
1. [Conclusión](#wrapup)

## Introducción <a class="anchor" id="intro"></a>
[Regresar al principio](#top)

Hasta el momento, debería tener varias soluciones y al menos una versión de la solución para cada una de ellas. Una vez creada una versión de la solución, es posible obtener recomendaciones de la misma y hacerse una idea de su comportamiento general.

Este cuaderno comienza con la implementación de cada una de las versiones de la solución del cuaderno anterior en campañas individuales. Una vez que están activas, hay recursos para consultar las recomendaciones y funciones de ayuda para transformar el resultado en algo más legible para el ser humano. 

Al igual que con su cliente en Amazon Personalize, puede modificar las funciones de ayuda para que se ajusten a la estructura de sus archivos de entrada de datos y así mantener el funcionamiento de la representación adicional.

Para empezar, una vez más, necesitamos importar bibliotecas, cargar valores de cuadernos anteriores y cargar el SDK.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd

In [ ]:
%store -r

In [ ]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Interactuar con campañas <a class="anchor" id="interact"></a>
[Regresar al principio](#top)

Ahora que todas las campañas se han implementado, podemos comenzar a recibir recomendaciones a través de una llamada de API. Cada una de las campañas se basa en una receta diferente, que se comporta de forma ligeramente distinta porque sirve para diferentes casos de uso. Cubriremos cada campaña en un orden diferente al que se utilizó en cuadernos anteriores, para tratar las posibles complejidades en orden ascendente (es decir, lo más sencillo primero).

En primer lugar, crearemos una función de apoyo para ayudar a dar sentido a los resultados devueltos por una campaña de personalización. Personalize solo devuelve un `item_id`. Esto es genial para mantener los datos de forma compacta, pero supone la necesidad de consultar una base de datos o una tabla de búsqueda para obtener un resultado legible para los cuadernos. Crearemos una función de ayuda para devolver un resultado legible para los humanos a partir del conjunto de datos LastFM.

Comience descargando el conjunto de datos que podemos utilizar para nuestra tabla de búsqueda.

In [ ]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
items_df.head(5)

Cuando definimos la columna ID como una columna índice, es común que devuelva un artista con tan solo buscar el ID.

In [ ]:
movie_id_example = 589
title = items_df.loc[movie_id_example]['title']
print(title)

Eso no es malo, pero se podría volver complicado tener que repetir esto en cada parte de nuestro código, por lo que la siguiente función facilitará esto.

In [ ]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

Ahora probemos con algunos valores simples para verificar nuestra detección de errores.

In [ ]:
# A known good id (The Princess Bride)
print(get_movie_by_id(movie_id="1197"))
# A bad type of value
print(get_movie_by_id(movie_id="987.9393939"))
# Really bad values
print(get_movie_by_id(movie_id="Steve"))

¡Genial! Ahora tenemos una forma de conseguir resultados. 

### SIMS

SIMS solo requiere un elemento como entrada y devolverá elementos con los que los usuarios interactúan de forma similar a su interacción con el elemento de entrada. En este caso concreto, el elemento es una película. 

Las celdas a continuación se encargarán de obtener las recomendaciones de SIMS y de mostrar los resultados. Veamos cuáles son las recomendaciones para el primer elemento que examinamos anteriormente en este cuaderno (Terminator 2: El juicio final).

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(589),
)

In [ ]:
item_list = get_recommendations_response['itemList']
for item in item_list:
    print(get_movie_by_id(movie_id=item['itemId']))

Felicidades, ¡esta es su primera lista de recomendaciones! Esta lista está bien, pero sería mejor ver las recomendaciones de nuestra colección de artistas de muestra en un marco de datos agradable. Una vez más, crearemos una función de ayuda para conseguirlo.

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

Ahora probaremos la función de ayuda con varias películas diferentes. Tomemos algunos datos de nuestro conjunto de datos para probar nuestra campaña SIMS. Seleccione 5 películas de nuestro marco de datos al azar.

Nota: Mostraremos títulos similares, por lo que es posible que desee volver a examinar la muestra hasta que reconozca algunas de las películas enumeradas.

In [ ]:
samples = items_df.sample(5)
samples

In [ ]:
sims_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, movie)

sims_recommendations_df

Es posible que observe que muchos de los elementos tienen el mismo aspecto, aunque es de esperar que no todos lo tengan (esto es más probable con un número menor de interacciones, lo que será más común con el pequeño conjunto de datos de movielens). Esto demuestra que las métricas de evaluación no deben ser lo único en lo que se basa la evaluación de la versión de la solución. Entonces, cuando esto sucede, ¿qué se puede hacer para mejorar los resultados?

Este es un buen momento para pensar en los hiperparámetros de las recetas de Personalize. La receta SIMS tiene un hiperparámetro `popularity_discount_factor` (ver [documentación](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html)). Aprovechar este hiperparámetro permite controlar el grado de precisión de los resultados. Este parámetro y su comportamiento será único para cada conjunto de datos que encuentre y depende de los objetivos de la empresa. Puede iterar sobre el valor de este hiperparámetro hasta que esté satisfecho con los resultados, o puede empezar aprovechando la función de optimización de hiperparámetros (HPO) de Personalize. Para obtener más información sobre el ajuste de los hiperparámetros y la optimización de parámetros, consulte el [documento](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

### Personalización del ususario

HRNN es uno de los algoritmos más avanzados que ofrece Amazon Personalize. Admite la personalización de los elementos para un usuario concreto en función de su comportamiento anterior y puede captar eventos en tiempo real a fin de modificar las recomendaciones para un usuario sin necesidad de volver a entrenarlo. 

Como la HRNN se basa en tener un muestreo de usuarios, carguemos los datos que necesitamos para ello y seleccionemos 3 usuarios al azar. Como MovieLens no incluye los datos de los usuarios, seleccionaremos 3 números aleatorios del rango de identificadores de usuario en el conjunto de datos.

In [ ]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

Ahora, presentamos las recomendaciones para los 3 usuarios seleccionados al azar. A continuación, exploraremos las interacciones en tiempo real antes de pasar a la clasificación personalizada.

Una vez más, creamos una función de ayuda para mostrar los resultados en un marco de datos agradable.

#### Resultados de las llamadas a la API

In [ ]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [ ]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

Aquí vemos claramente que las recomendaciones para cada usuario son diferentes. Si necesitara un caché para estos resultados, podría empezar por ejecutar las llamadas a la API a través de todos sus usuarios y almacenar los resultados, o bien podría utilizar una exportación por lotes, que se tratará más adelante en este cuaderno.

Ahora vamos a aplicar los filtros de elementos para ver las recomendaciones de uno de estos usuarios dentro de un género


In [ ]:
def get_new_recommendations_df_by_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

Puede ver las recomendaciones de películas dentro de un género determinado. Dentro de una aplicación de video bajo demanda (VOD), se pueden crear estantes (también conocidos como carril o carrusel) fácilmente por medio de estos filtros. Dependiendo de la información que tenga sobre sus elementos, también podría filtrar por información adicional como una palabra clave, el año/década, etc.

In [ ]:
recommendations_df_shelves = pd.DataFrame()
for filter_arn in meta_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)
for filter_arn in decade_filter_arns:
    recommendations_df_shelves = get_new_recommendations_df_by_filter(recommendations_df_shelves, user, filter_arn)

recommendations_df_shelves

El siguiente tema es el de los eventos en tiempo real. Personalize tiene la capacidad de escuchar los eventos de su aplicación para actualizar las recomendaciones mostradas al usuario. Esto es especialmente útil en las cargas de trabajo de los medios de comunicación, como el video bajo demanda, donde la intención de un cliente puede variar en función de si lo está viendo con sus hijos o solo.

Además, los eventos que se registran a través de este sistema se almacenan hasta que se produce una llamada de eliminación por su parte y se utilizan como datos históricos junto con los demás datos de interacción que ha proporcionado cuando entrena sus próximos modelos.

#### Eventos en tiempo real

Comience por crear un rastreador de eventos que se adjunte a la campaña.

In [ ]:
response = personalize.create_event_tracker(
    name='MovieTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

Crearemos un código que simule la interacción de un usuario con un elemento determinado. Después de ejecutar este código, obtendrá recomendaciones que se diferencian de los resultados anteriores.

Comenzamos creando algunos métodos para la simulación de eventos en tiempo real.

In [ ]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

Hasta ahora no hemos generado ningún evento en tiempo real, solo hemos configurado el código. Con el fin de comparar las recomendaciones antes y después de los eventos en tiempo real, seleccionemos un usuario y generemos las recomendaciones originales para él.

In [ ]:
# First pick a user
user_id = user

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendation_list.append(artist)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [user_id])
user_recommendations_df

Ahora tenemos una lista de recomendaciones para este usuario antes de aplicar cualquier evento en tiempo real. Ahora elegiremos 3 artistas al azar con los que simularemos que nuestro usuario interactúa y veremos cómo cambian las recomendaciones.

In [ ]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [ ]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, movie,'click')
    time.sleep(5)

Ahora podemos ver cómo los eventos de clic cambiaron las recomendaciones.

In [ ]:
user_recommendations_df

En la celda anterior, la primera columna después del índice muestra las recomendaciones por defecto del usuario desde la personalización del usuario y cada columna después de eso tiene un encabezado del artista con el que interactuó a través de un evento en tiempo real y las recomendaciones después de que este evento ocurriera. 

Es posible que el comportamiento no cambie mucho. Esto se debe a la naturaleza relativamente limitada de este conjunto de datos y al efecto de unos pocos clics aleatorios. Si desea comprender mejor esto, intente simular que hace clic en más películas y debería ver un impacto más pronunciado.

Ahora veamos los filtros de eventos, que permiten filtrar los elementos en función de los datos de interacción. Para este conjunto de datos, podría tratarse de hacer clic o de ver en función de los datos que importamos, pero podría basarse en cualquier esquema de interacción que diseñe (hacer clic, valorar, me gusta, ver, comprar, etc.) En el caso de las estanterías de VOD, se podría mover un título de "Recomendados para ti" a "Ver otra vez", las recomendaciones para ver otra vez se basarán en las interacciones actuales de los usuarios, pero solo recomendarán títulos que ya hayan sido vistos.


In [ ]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

Ahora enviaremos un evento de ver para 4 recomendaciones no vistas, lo que simularía ver 4 películas. En una aplicación VOD, puede optar por enviar un evento después de que hayan visto una cantidad significativa (más del 75 %) de un contenido. El envío al 100 % completo podría pasar por alto a las personas que se detienen antes de los créditos.

In [ ]:
 # Get the recommendations
top_unwatched_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = userpersonalization_campaign_arn,
    userId = str(user_id),
    filterArn = filter_arn,
    numResults=4)
item_list = top_unwatched_recommendations_response['itemList']
for item in item_list:
    print('sending event watch for ' + get_movie_by_id(item['itemId']))
    send_movie_click(USER_ID=user_id, ITEM_ID=item['itemId'], EVENT_TYPE='watch')
    time.sleep(10)

Ahora podemos mirar los filtros de eventos para ver las recomendaciones actualizadas, tanto las vistas como las no vistas. 

In [ ]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

### Clasificación personalizada

El caso de uso principal de la clasificación personalizada es tomar una colección de elementos y presentarlos en orden de prioridad o de interés probable para un usuario. Para una aplicación de VOD, se desea obtener dinámicamente un estante/carril/carrusel personalizado basado en alguna información (director, ubicación, franquicia de superhéroes, período de tiempo de la película, etc.). Es posible que no tenga esta información en sus metadatos, por lo que un filtro de metadatos de artículos no funcionará. Sin embargo, puede tener esta información dentro de su sistema para generar la lista de artículos. 

A modo de demostración, utilizaremos el mismo usuario de antes y una colección aleatoria de artículos.

In [ ]:
rerank_user = user
rerank_items = items_df.sample(25).index.tolist()

Ahora, cree un marco de datos agradable que muestre los datos de entrada.

In [ ]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

Luego, haga la llamada a la API de clasificación personalizada.

In [ ]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

Ahora, agregue los elementos reordenados como una segunda columna al marco de datos original, para una comparación en paralelo.

In [ ]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

Puede ver anteriormente cómo se reordenó cada entrada en función de la comprensión del usuario por parte del modelo. Esta es una tarea muy popular cuando se tiene una colección de elementos para mostrar a un usuario, una lista de promociones por ejemplo.

## Recomendaciones por lote <a class="anchor" id="batch"></a>
[Regresar al principio](#top)

Existen muchos casos en los que quizá desee tener un mayor conjunto de datos de recomendaciones exportadas. Recientemente, Amazon Personalize lanzó recomendaciones por lote como una forma de exportar una colección de recomendaciones a S3. En este ejemplo, le explicaremos cómo hacer esto para la solución de HRNN. Para obtener más información sobre las recomendaciones por lote, consulte la [documentación](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch). Esta característica se aplica a todas las recetas, pero el formato de salida variará.

Una implementación sencilla tiene la siguiente apariencia:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

Se ha determinado la importación SDK, la versión de la solución de arn y los roles de arn. Esto solo deja una entrada, una salida y un nombre de trabajo por definir.

Empezando por la entrada para HRNN, se ve así:


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

Esto debería generar una salida con la siguiente apariencia:

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

La salida es un archivo JSON Lines. Consiste en objetos JSON individuales, uno por línea. Así que luego necesitaremos trabajar más para asimilar los resultados en este formato.

### Creación del archivo de entrada

Cuando utiliza la característica de lote, usted especifica los usuarios que desea que reciban recomendaciones para el momento en el que se haya completado el trabajo. La celda que aparece a continuación seleccionará nuevamente y de forma aleatoria a algunos usuarios y, luego, creará el archivo y lo guardará en el disco. A partir de allí, lo cargará al S3 para utilizarlo luego en la llamada a la API.

In [ ]:
# We will use the same users from before
users
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [ ]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

Cargue el archivo a S3 y guarde el trayecto como una variable para más tarde.

In [ ]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

Las recomendaciones por lote leen la entrada del archivo que hemos subido al S3. De forma similar, las recomendaciones por lote guardarán el archivo de salida en el S3. Así que definimos el trayecto de salida donde se deberían guardar los resultados.

In [ ]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

Ahora solo hagamos la llamada para poner en marcha el proceso de exportación del lote.

In [ ]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = userpersonalization_solution_version_arn,
    jobName = "VOD-POC-Batch-Inference-Job-UserPersonalization_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Ejecute el bucle while a continuación para hacer un seguimiento del estado de la llamada de recomendación por lote. Esto puede llevar alrededor de 30 minutos en completarse debido a que Personalize necesita soportar la infraestructura para llevar a cabo la tarea. Estamos probando la característica con un conjunto de datos de solo 3 usuarios, lo que no es un uso eficiente de este mecanismo. Normalmente, solo utilizaría esta característica para el procesamiento en masa, caso en el que las eficiencias se volverán claras.

In [ ]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_movie_by_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

## Conclusión <a class="anchor" id="wrapup"></a>
[Regresar al principio](#top)

Con esto, ya tiene una colección de modelos totalmente operativa para abordar varios escenarios de recomendación y personalización, así como las habilidades para manipular los datos de los clientes y así integrarlos mejor con el servicio, y un conocimiento de cómo hacer todo esto a través de las API y aprovechando las herramientas de ciencia de datos de código abierto.

Utilice estos cuadernos como guía para empezar a trabajar con sus clientes en los POC. Si descubre componentes que faltan o nuevos enfoques, envíe una solicitud de acceso y proporcione cualquier componente útil adicional que pueda faltar en esta colección.

Deberá asegurarse de limpiar todos los recursos implementados durante esta POC. Hemos proporcionado un cuaderno separado que muestra cómo identificar y eliminar los recursos en `06_Clean_Up_Resources.ipynb`.

In [ ]:
%store event_tracker_arn
%store batchInferenceJobArn